<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Lab: Fun with Neural Nets

---

Below is a procedure for building a neural network to recognize handwritten digits.  The data is from [Kaggle](https://www.kaggle.com/c/digit-recognizer/data), and you will submit your results to Kaggle to test how well you did!

1. Load the training data (`train.csv`) from [Kaggle](https://www.kaggle.com/c/digit-recognizer/data) (click on the link, be sure to sign in, then scroll down and on the left-hand side, click "Download All")
2. Set up X and y (feature matrix and target vector)
3. Split X and y into train and test subsets.
4. Preprocess your data

   - When dealing with image data, you need to normalize your `X` by dividing each value by the max value of a pixel (255).
   - Since this is a multiclass classification problem, keras needs `y` to be a one-hot encoded matrix
   
5. Create your network.

   - Remember that for multi-class classification you need a softamx activation function on the output layer.
   - You may want to consider using regularization or dropout to improve performance.
   
6. Train your network.
7. If you are unhappy with your model performance, try to tighten up your model by adding hidden layers, adding hidden layer units, changing the activation functions on the hidden layers, etc.
8. Load in [Kaggle's](https://www.kaggle.com/c/digit-recognizer/data) `test.csv`
9. Create your predictions (these should be numbers in the range 0-9).
10. Save your predictions and submit them to Kaggle.

---

For this lab, you should complete the above sequence of steps for _at least_ two of the four "configurations":

1. Using a `tensorflow` network (we did _not_ cover this in class!)
2. Using a `keras` convolutional network
3. Using a `keras` network with regularization
4. Using a `tensorflow` convolutional network (we did _not_ cover this in class!)

In [103]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import utils
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout

np.random.seed(42)

In [70]:
train = pd.read_csv('./digit-recognizer/train.csv')
test = pd.read_csv("./digit-recognizer/test.csv")

In [71]:
train

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [72]:
# X, y
X = train.drop(columns='label') # Feature matrix
y = train['label'] # Target vector

In [73]:
# Each row is an image
# Each image is 28 x 28 so in total 784 pixels
X.shape

(42000, 784)

In [74]:
# Train, test, split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [75]:
# Make sure each value is a float. (Otherwise, we get an error.)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# The current range of X_train and X_test is 0 to 255.
# The code below is equivalent to X_train = X_train / 255.
# This scales each value to be between 0 and 1.

X_train /= 255
X_test /= 255
test /= 255 #normalize 

In [76]:
# Reshape
X_train = X_train.values.reshape(-1,28,28,1)
X_test = X_test.values.reshape(-1,28, 28, 1)
test = test.values.reshape(-1, 28, 28, 1)

In [77]:
# Check shape of full training data.
X_train.shape

(31500, 28, 28, 1)

In [78]:
# Change y_train.
y_train = utils.to_categorical(y_train)
y_test = utils.to_categorical(y_test)

### 2. Using a keras convolutional network

In [91]:
# 7. 0.98635

# Instantiate the Model
model = Sequential()
model.add(Conv2D(filters = 32, 
                kernel_size = (5,5), 
                activation = 'relu', 
                input_shape=(28, 28, 1)))

model.add(MaxPooling2D
             (pool_size=(2,2))) # dimensions of region of pooling
model.add(Conv2D(filters = 32,
                     kernel_size= (5,5),
                     activation= 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

history = model.fit(X_train,
                        y_train,
                        batch_size=256,
                        validation_data=(X_test, y_test),
                        epochs=20,
                        verbose=1)

Epoch 1/20
124/124 [==============================] - 17s 134ms/step - loss: 0.5392 - accuracy: 0.8420 - val_loss: 0.1529 - val_accuracy: 0.9540
Epoch 2/20
124/124 [==============================] - 22s 180ms/step - loss: 0.1197 - accuracy: 0.9643 - val_loss: 0.1041 - val_accuracy: 0.9670
Epoch 3/20
124/124 [==============================] - 15s 118ms/step - loss: 0.0757 - accuracy: 0.9771 - val_loss: 0.1074 - val_accuracy: 0.9654
Epoch 4/20
124/124 [==============================] - 15s 123ms/step - loss: 0.0623 - accuracy: 0.9801 - val_loss: 0.0578 - val_accuracy: 0.9825
Epoch 5/20
124/124 [==============================] - 22s 177ms/step - loss: 0.0465 - accuracy: 0.9863 - val_loss: 0.0627 - val_accuracy: 0.9807
Epoch 6/20
124/124 [==============================] - 19s 150ms/step - loss: 0.0395 - accuracy: 0.9877 - val_loss: 0.0530 - val_accuracy: 0.9832
Epoch 7/20
124/124 [==============================] - 18s 145ms/step - loss: 0.0338 - accuracy: 0.9892 - val_loss: 0.0525 - val_ac

In [92]:
# predict results
results = model.predict(test)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [93]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("submission.csv",index=False)

### 3. Using a keras network with regularization

In [105]:
# Credit to David Lee for assistance
model = Sequential()
model.add(Flatten(input_shape=(28, 28, 1))) # This is flattening our matrix into a vector.
model.add(Dense(128, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))
# Compile model.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# Fit model on training data.
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1, mode='auto')
history = model.fit(X_train,
                    y_train, 
                    batch_size=256,
                    validation_data=(X_test, y_test),
                    epochs=50,
                    verbose=1,
                    callbacks= [early_stop])

Epoch 1/50
124/124 [==============================] - 1s 6ms/step - loss: 0.8676 - accuracy: 0.7308 - val_loss: 0.3313 - val_accuracy: 0.9056
Epoch 2/50
124/124 [==============================] - 0s 4ms/step - loss: 0.3802 - accuracy: 0.8888 - val_loss: 0.2482 - val_accuracy: 0.9279
Epoch 3/50
124/124 [==============================] - 1s 4ms/step - loss: 0.3069 - accuracy: 0.9098 - val_loss: 0.2075 - val_accuracy: 0.9390
Epoch 4/50
124/124 [==============================] - 0s 4ms/step - loss: 0.2619 - accuracy: 0.9224 - val_loss: 0.1775 - val_accuracy: 0.9481
Epoch 5/50
124/124 [==============================] - 0s 4ms/step - loss: 0.2325 - accuracy: 0.9311 - val_loss: 0.1615 - val_accuracy: 0.9537
Epoch 6/50
124/124 [==============================] - 1s 5ms/step - loss: 0.2079 - accuracy: 0.9376 - val_loss: 0.1497 - val_accuracy: 0.9559
Epoch 7/50
124/124 [==============================] - 0s 4ms/step - loss: 0.1919 - accuracy: 0.9420 - val_loss: 0.1411 - val_accuracy: 0.9582
Epoch 

In [106]:
# submitting predictions from https://www.kaggle.com/yassineghouzam/introduction-to-cnn-keras-0-997-top-6
# predict results
results = model.predict(test)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [107]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("submission_1.csv",index=False)